In [ ]:
# !pip install -q -U torch datasets sentence_transformers
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers
!pip install huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.0 requires accelerate>=0.34.0, but you have accelerate 0.27.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.0 requires accelerate>=0.34.0, but you have accelerate 0.27.1 which is incompatible.
trl 0.12.0 requires datasets>=2.21.0, but you have datasets 2.17.0 which is incompatible.


## Importing Libraries Required

In [ ]:
import os
import transformers
from google.colab import userdata
import torch
from trl import SFTTrainer
from google.colab import userdata
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
test_df = pd.read_csv('test.csv')

## Configurations for LORA and Quantization

In [ ]:
model_name='google/gemma-2b-it'

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load pre-trained config
#################################################################

tokenizer = AutoTokenizer.from_pretrained(model_name,token = userdata.get('HF_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"":0},token = userdata.get('HF_TOKEN'))

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Loading the dataset

In [ ]:
from datasets import load_dataset

data = load_dataset("microsoft/orca-math-word-problems-200k")
data = data.map(lambda samples: tokenizer(samples["question"]), batched=True)

Map:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask'],
        num_rows: 200035
    })
})

## Model

In [ ]:
def formatting_func(example):
    text = f"Question: {example['question'][0]}\nAnswer: {example['answer'][0]}"
    return [text]
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=2,
        max_steps=100,
        learning_rate=0.001,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        # gradient_checkpointing=True # Enable gradient checkpointing

    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))


trainable model parameters: 9805824
all model parameters: 1525073920
percentage of trainable model parameters: 0.64%


## Model Training

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: raghavanb (raghavanb-student). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.543400
20,1.934400
30,1.848200
40,1.780000
50,1.781900
60,1.759500
70,1.791000
80,1.671700
90,1.646400
100,1.632400


TrainOutput(global_step=100, training_loss=1.8388760948181153, metrics={'train_runtime': 272.4877, 'train_samples_per_second': 2.936, 'train_steps_per_second': 0.367, 'total_flos': 598403332608000.0, 'train_loss': 1.8388760948181153, 'epoch': 0.003999300122478566})

## Model Inference

In [ ]:
## inference
from transformers import pipeline
pipeline = pipeline(
    'text-generation',
    model = model,
    tokenizer = AutoTokenizer.from_pretrained(model_name),
    device_map = 'auto',
)



In [ ]:
sequences = pipeline(
    'Question: John and Mary have three kids. The sum of their ages is 22. The first son is 5 years old. The eldest has double the age of the first son. What is the age of the remaining son??\nAnswer:',
    max_length = 1000,
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1,
    truncation = True
)
sequences[0]['generated_text']

"Question: John and Mary have three kids. The sum of their ages is 22. The first son is 5 years old. The eldest has double the age of the first son. What is the age of the remaining son??\nAnswer: 11 years old.\nExplanation: The first son's age is some value. The second son's age is twice the value of the first son. The third son's age is 5 years old. The sum of the ages of all three children is 22. Therefore, the remaining son has a specific age."